In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
#https://www.kaggle.com/datasets/snehangsude/audible-dataset

In [3]:
wd = os.getcwd()+"/"
audible_uncleaned = pd.read_csv(wd+"audible_uncleaned.csv")

In [4]:
audible_uncleaned.head()

,name,author,narrator,time,releasedate,language,stars,price
0,Geronimo Stilton #11 & #12,Writtenby:GeronimoStilton,Narratedby:BillLobely,2 hrs and 20 mins,04-08-08,English,5 out of 5 stars34 ratings,468.00
1,The Burning Maze,Writtenby:RickRiordan,Narratedby:RobbieDaymond,13 hrs and 8 mins,01-05-18,English,4.5 out of 5 stars41 ratings,820.00
2,The Deep End,Writtenby:JeffKinney,Narratedby:DanRussell,2 hrs and 3 mins,06-11-20,English,4.5 out of 5 stars38 ratings,410.00
3,Daughter of the Deep,Writtenby:RickRiordan,Narratedby:SoneelaNankani,11 hrs and 16 mins,05-10-21,English,4.5 out of 5 stars12 ratings,615.00
4,"The Lightning Thief: Percy Jackson, Book 1",Writtenby:RickRiordan,Narratedby:JesseBernstein,10 hrs,13-01-10,English,4.5 out of 5 stars181 ratings,820.00


In [5]:
#name. Number of books can be extracted depending on the case
#author: remove written by and separate names
#narrator: idem
#time:convert to datetime64
#releasedate: convert to datetime 64
#language: --
#stars: separate stars and ratings
#price: convert to euros.

In [6]:
audible_uncleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         87489 non-null  object
 1   author       87489 non-null  object
 2   narrator     87489 non-null  object
 3   time         87489 non-null  object
 4   releasedate  87489 non-null  object
 5   language     87489 non-null  object
 6   stars        87489 non-null  object
 7   price        87489 non-null  object
dtypes: object(8)
memory usage: 5.3+ MB


In [8]:
audible_cleaned =pd.DataFrame()

In [9]:
audible_cleaned["name"]=audible_uncleaned["name"].copy()

In [15]:
audible_cleaned["author"]= audible_uncleaned["author"]\
                                                        .str.extract(r"Writtenby:(.*)")

In [20]:
audible_cleaned

,name,author
0,Geronimo Stilton #11 & #12,GeronimoStilton
1,The Burning Maze,RickRiordan
2,The Deep End,JeffKinney
3,Daughter of the Deep,RickRiordan
4,"The Lightning Thief: Percy Jackson, Book 1",RickRiordan
...,...,...
87484,Last Days of the Bus Club,ChrisStewart
87485,The Alps,StephenO'Shea
87486,The Innocents Abroad,MarkTwain
87487,A Sentimental Journey,LaurenceSterne


In [48]:
""" def splitCamelCase(str):
    def addAsteriskInCaseChange(str):
        new_string="*"
        for i in str:
            if(i.isupper() and new_string[-1].isalpha()):
                new_string+="*"+i
            else:
                new_string+=i
        return new_string

    def splitStrOnAsterisk(str):
        return str.split("*")
    def removeEmpty(lst):
    #if '' in lst:
    #    x.remove('')
        return [n for n in str if not(n=='')]
    def joinListWithSpaces(lst):
        return " ".join(lst)
    return joinListWithSpaces(removeEmpty(splitStrOnAsterisk( addAsteriskInCaseChange(str) ))) """

In [52]:
def splitCamelCase(str):
 
    new_string="*"
    for i in str:
        if(i.isupper() and new_string[-1].isalpha()):
            new_string+="*"+i
        else:
            new_string+=i
    
    x= new_string.split("*")
    if '' in x:
       x.remove('')
    return " ".join(x)


In [53]:
audible_cleaned["author"].apply(splitCamelCase)

0        Geronimo Stilton
1            Rick Riordan
2             Jeff Kinney
3            Rick Riordan
4            Rick Riordan
               ...       
87484       Chris Stewart
87485      Stephen O'Shea
87486          Mark Twain
87487     Laurence Sterne
87488      Mark Kurlansky
Name: author, Length: 87489, dtype: object

In [62]:
audible_cleaned["narrator"] = audible_uncleaned["narrator"]\
                                                            .str.extract(r"Narratedby:(.*)")

In [63]:
audible_cleaned["narrator"]=audible_cleaned["narrator"].apply(splitCamelCase)

In [64]:
audible_cleaned

,name,author,narrator
0,Geronimo Stilton #11 & #12,GeronimoStilton,Bill Lobely
1,The Burning Maze,RickRiordan,Robbie Daymond
2,The Deep End,JeffKinney,Dan Russell
3,Daughter of the Deep,RickRiordan,Soneela Nankani
4,"The Lightning Thief: Percy Jackson, Book 1",RickRiordan,Jesse Bernstein
...,...,...,...
87484,Last Days of the Bus Club,ChrisStewart,Chris Stewart
87485,The Alps,StephenO'Shea,Robert Fass
87486,The Innocents Abroad,MarkTwain,Flo Gibson
87487,A Sentimental Journey,LaurenceSterne,Anton Lesser


In [67]:
pd.to_timedelta(audible_uncleaned["time"])

ValueError: invalid unit abbreviation: hrsand